In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

In [3]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10948908292365568286
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9399762944
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15725146134698470536
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:22:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 1745276110
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12549827829692726367
 physical_device_desc: "device: 1, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 2144165316]

In [40]:
import pandas as pd
#import pandas_profiling
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import optuna.integration.lightgbm as lgb
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier


print("OK\n")

OK



In [44]:
app_train =  pd.read_csv('./home-credit-default-risk/exports/app_train_res.csv')
app_test =  pd.read_csv('./home-credit-default-risk/exports/app_test_res.csv')
# 他社債務
#hc_br_train = pd.read_csv('./home-credit-default-risk/exports/hc_br_train_axis-60.csv')
#hc_br_test = pd.read_csv('./home-credit-default-risk/exports/hc_br_test_axis-60.csv')
# hc Cash loans/Cash loans
#hc_only_cash_cash_train = pd.read_csv('./home-credit-default-risk/datasets/app_train_hc_cash-cash-domein_axis-24-monthes.csv')
#hc_only_cash_cash_test = pd.read_csv('./home-credit-default-risk/datasets/app_test_hc_cash-cash-domein_axis-24-monthes.csv')
# Revolving loans/Consumer loans
#hc_only_rev_cons_train = pd.read_csv('./home-credit-default-risk/datasets/app_train_hc_cash-cons_axis-24-monthes.csv')
#hc_only_rev_cons_test = pd.read_csv('./home-credit-default-risk/datasets/app_test_hc_cash-cons_axis-24-monthes.csv')
# hc  Revolving loans/Cash loans

app_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170,...,0,0,0,0,0,0,1,0,1,0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118,...,0,0,0,0,0,0,0,0,0,0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175,...,0,0,0,0,0,0,0,0,0,0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000,...,0,0,0,0,0,1,0,0,1,0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000,...,0,0,0,0,0,0,0,0,0,0


In [6]:
columns = ['TARGET']
#hc_br_train.drop(columns, axis=1, inplace=True)
#hc_only_cash_cash_train.drop(columns, axis=1, inplace=True)
#hc_only_rev_cons_train.drop(columns, axis=1, inplace=True)
#hc_only_consumer_train.drop(columns, axis=1, inplace=True)
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,...,0,0,0,0,0,0,1,0,1,0
1,100003,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,...,0,1,0,0,0,0,0,0,1,0
2,100004,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# trainへマージ
#app_train = pd.merge(app_train, hc_br_train, on='SK_ID_CURR', how='left')
# testへマージ
#app_test = pd.merge(app_test, hc_br_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [10]:
# trainへマージ
#app_train = pd.merge(app_train, hc_only_cash_cash_train, on='SK_ID_CURR', how='left')
# testへマージ
#app_test = pd.merge(app_test, hc_only_cash_cash_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [11]:
# trainへマージ
#app_train = pd.merge(app_train, hc_only_rev_cons_train, on='SK_ID_CURR', how='left')
# testへマージ
#app_test = pd.merge(app_test, hc_only_rev_cons_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [12]:
# trainへマージ
#app_train = pd.merge(app_train, hc_only_consumer_train, on='SK_ID_CURR', how='left')
# testへマージ
#app_test = pd.merge(app_test, hc_only_consumer_test, on='SK_ID_CURR', how='left')
#len(app_train)

In [33]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

# 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(X_train[num_cols])

# 変換後のデータで各列を置換
#X_train[num_cols] = pt.transform(X_train[num_cols])
#X_test[num_cols] = pt.transform(X_test[num_cols])

X_train.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,0,0,0,0,0,0,1,0,1,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0,1,0,0,0,0,0,0,1,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,0,0,0,0,0,0,0,0,0,0


## 特徴量の重要度

In [34]:
dtrain = xgb.DMatrix(X_train, label=y_train)
params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}
num_round = 50
model = xgb.train(params, dtrain, num_round)

# 重要度の上位を出力する
fscore = model.get_score(importance_type = 'total_gain')
fscore = sorted([(k, v) for k, v in fscore.items()], key=lambda tpl: tpl[1], reverse=True)
len(fscore)

[23:09:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




207

In [36]:
fscore[:50]

[('EXT_SOURCE_3', 28587.6171875),
 ('EXT_SOURCE_2', 21610.326171875),
 ('EXT_SOURCE_1', 11949.498046875),
 ('OWN_CAR_AGE', 3901.67822265625),
 ('AMT_GOODS_PRICE', 3522.70068359375),
 ('AMT_CREDIT', 2935.35888671875),
 ('DAYS_EMPLOYED', 2801.556396484375),
 ('AMT_ANNUITY', 2771.530029296875),
 ('DAYS_BIRTH', 2641.04736328125),
 ('DAYS_ID_PUBLISH', 2056.57666015625),
 ('CODE_GENDER_M', 1636.694580078125),
 ('DAYS_LAST_PHONE_CHANGE', 1566.266357421875),
 ('NAME_EDUCATION_TYPE_Higher education', 1357.1409912109375),
 ('NAME_EDUCATION_TYPE_Secondary / secondary special', 1205.5501708984375),
 ('DAYS_REGISTRATION', 1090.89306640625),
 ('AMT_INCOME_TOTAL', 954.6192016601562),
 ('SK_ID_CURR', 850.0956420898438),
 ('CODE_GENDER_F', 805.8289794921875),
 ('LIVINGAPARTMENTS_MODE', 801.6151733398438),
 ('LANDAREA_MODE', 801.5758056640625),
 ('YEARS_BEGINEXPLUATATION_MODE', 787.3121948242188),
 ('NAME_INCOME_TYPE_Working', 758.7926025390625),
 ('FLAG_DOCUMENT_3', 758.1090087890625),
 ('AMT_REQ_CREDI

In [26]:
type(fscore)

list

In [27]:
test_columns = []
for i in range(50):
    test_columns.append(fscore[i][0])
test_columns.append('SK_ID_CURR')

In [28]:
train_columns = test_columns.copy()
train_columns.append('TARGET')

In [29]:
app_train = app_train[train_columns].copy()
app_test = app_test[test_columns].copy()
app_train.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,OWN_CAR_AGE,AMT_GOODS_PRICE,AMT_CREDIT,DAYS_EMPLOYED,AMT_ANNUITY,DAYS_BIRTH,DAYS_ID_PUBLISH,...,AMT_REQ_CREDIT_BUREAU_DAY,YEARS_BUILD_MODE,NONLIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,YEARS_BEGINEXPLUATATION_MEDI,COMMONAREA_MEDI,BASEMENTAREA_MEDI,DEF_30_CNT_SOCIAL_CIRCLE,SK_ID_CURR,TARGET
0,0.139376,0.262949,0.083037,19.702795,351000.0,406597.5,-637.0,24700.5,-9461.0,-2120.0,...,0.000000,0.634100,0.000000,0.019000,0.972200,0.014400,0.036900,2.0,100002,1.0
1,0.453866,0.622246,0.311267,16.325264,1129500.0,1293502.5,-1188.0,35698.5,-16765.0,-291.0,...,0.000000,0.804000,0.000000,0.054900,0.985100,0.060800,0.052900,0.0,100003,0.0
2,0.729567,0.555912,0.631110,26.000000,135000.0,135000.0,-225.0,6750.0,-19046.0,-2531.0,...,0.000000,0.760966,0.007968,0.107398,0.977789,0.044521,0.087798,0.0,100004,0.0
3,0.553797,0.650442,0.654638,12.254344,297000.0,312682.5,-3039.0,29686.5,-19005.0,-2437.0,...,0.007323,0.760868,0.007957,0.107290,0.977844,0.044321,0.087773,0.0,100006,0.0
4,0.556818,0.322738,0.607932,14.425593,513000.0,513000.0,-3038.0,21865.5,-19932.0,-3458.0,...,0.000000,0.761285,0.008205,0.107298,0.977632,0.044647,0.087845,0.0,100007,0.0


## 学習

In [45]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()


X_train.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,0,0,0,0,0,0,1,0,1,0
1,100003,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0,1,0,0,0,0,0,0,1,0
2,100004,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

[23:16:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.73071	eval-auc:0.71510
[1]	train-auc:0.74164	eval-auc:0.72239
[2]	train-auc:0.74886	eval-auc:0.72806
[3]	train-auc:0.75421	eval-auc:0.73024
[4]	train-auc:0.75893	eval-auc:0.73311
[5]	train-auc:0.76105	eval-auc:0.73403
[6]	train-auc:0.76360	eval-auc:0.73412
[7]	train-auc:0.76501	eval-auc:0.73488
[8]	train-auc:0.76667	eval-auc:0.73573
[9]	train-auc:0.76795	eval-auc:0.73625
[10]	train-auc:0.76982	eval-auc:0.73655
[11]	train-auc:0.77132	eval-auc:0.73665
[12]	train-auc:0.77311	eval-auc:0.73719
[13]	t

[8]	train-auc:0.76805	eval-auc:0.73378
[9]	train-auc:0.76968	eval-auc:0.73433
[10]	train-auc:0.77145	eval-auc:0.73480
[11]	train-auc:0.77285	eval-auc:0.73491
[12]	train-auc:0.77418	eval-auc:0.73527
[13]	train-auc:0.77688	eval-auc:0.73592
[14]	train-auc:0.77803	eval-auc:0.73602
[15]	train-auc:0.77974	eval-auc:0.73689
[16]	train-auc:0.78108	eval-auc:0.73683
[17]	train-auc:0.78244	eval-auc:0.73711
[18]	train-auc:0.78332	eval-auc:0.73726
[19]	train-auc:0.78497	eval-auc:0.73742
[20]	train-auc:0.78612	eval-auc:0.73802
[21]	train-auc:0.78776	eval-auc:0.73837
[22]	train-auc:0.78901	eval-auc:0.73835
[23]	train-auc:0.79050	eval-auc:0.73859
[24]	train-auc:0.79273	eval-auc:0.73973
[25]	train-auc:0.79409	eval-auc:0.74031
[26]	train-auc:0.79538	eval-auc:0.74121
[27]	train-auc:0.79646	eval-auc:0.74111
[28]	train-auc:0.79852	eval-auc:0.74154
[29]	train-auc:0.80000	eval-auc:0.74210
[30]	train-auc:0.80142	eval-auc:0.74252
[31]	train-auc:0.80280	eval-auc:0.74270
[32]	train-auc:0.80400	eval-auc:0.74291
[3

In [47]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [48]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [49]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [50]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [51]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [52]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [53]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.066431,0.070727,0.039247,0.088131,0.054288
1,0.163498,0.154405,0.140545,0.157096,0.216298
2,0.035659,0.024244,0.022444,0.028427,0.043893
3,0.044009,0.049787,0.042694,0.037918,0.056514
4,0.141837,0.149283,0.135018,0.151844,0.166496


In [54]:
submit_all.corr()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
submit_all_00,1.000000,0.930574,0.929197,0.932876,0.927710
submit_all_01,0.930574,1.000000,0.931142,0.930891,0.922262
submit_all_02,0.929197,0.931142,1.000000,0.931972,0.928541
submit_all_03,0.932876,0.930891,0.931972,1.000000,0.923634
submit_all_04,0.927710,0.922262,0.928541,0.923634,1.000000


In [55]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.063765
1,100005,0.166368
2,100013,0.030933
3,100028,0.046184
4,100038,0.148896


In [56]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_all_xgb.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.012753
1,100005,0.033274
2,100013,0.006187
3,100028,0.009237
4,100038,0.029779


In [57]:
submit_all.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64